In [2]:
import torch
from sklearn.cluster import MiniBatchKMeans
import numpy as np

In [3]:
state_dict = torch.load('output_DEPRECATED/pytorch_model.bin')

In [14]:
def count_param(dict_):
    num_param = sum([x.numel() for x in dict_.values()])
    print("{:,}".format(num_param))
    return num_param

def quantization(state_dict, n_clusters=128):
    flattened = torch.cat([param.flatten() for param in state_dict.values()]).cpu().numpy().reshape(-1, 1)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, max_no_improvement=2).fit(flattened)
    clustered = [kmeans.cluster_centers_[index] for index in kmeans.predict(flattened)]

    # replace the original parameters with cluster centers
    cursor = 0
    for key in state_dict.keys():
        shape = state_dict[key].shape
        size = state_dict[key].numel()
        state_dict[key] = torch.Tensor(clustered[cursor:cursor + size]).reshape(shape)
        cursor += size
        
    return state_dict

In [7]:
dict_ = { key: state_dict[key]
          for key in list(state_dict.keys())[:1] }

In [15]:
_ = count_param(dict_)

23,440,896


In [129]:
%%time
_ = quantization(dict_)

CPU times: user 1min 13s, sys: 580 ms, total: 1min 14s
Wall time: 1min 14s
